In [ ]:
from azureml.core import Workspace, Experiment

ws = Workspace.get(name="udacity-project")
exp = Experiment(workspace=ws, name="udacity-project")

print('Workspace name: ' + ws.name, 
      'Azure region: ' + ws.location, 
      'Subscription id: ' + ws.subscription_id, 
      'Resource group: ' + ws.resource_group, sep = '\n')

run = exp.start_logging()

In [ ]:
from azureml.core.compute import ComputeTarget, AmlCompute

# TODO: Create compute cluster
# Use vm_size = "Standard_D2_V2" in your provisioning configuration.
# max_nodes should be no greater than 4.


# assuming cluster name does not already exist, provision for cluster
name = "cpu-cluster-compute"
compute_config = AmlCompute.provisioning_configuration(
    vm_size='Standard_D2_V2', 
    vm_priority='lowpriority', 
    max_nodes=4
)
cpu_cluster = ComputeTarget.create(ws, name, compute_config)
cpu_cluster.wait_for_completion(show_output=True)

In [ ]:
# create training directory
if "training" not in os.listdir():
    os.mkdir("./training")

In [ ]:
!cp train.py ./training

In [ ]:
from azureml.widgets import RunDetails
from azureml.train.sklearn import SKLearn
from azureml.train.hyperdrive.run import PrimaryMetricGoal
from azureml.train.hyperdrive.policy import BanditPolicy
from azureml.train.hyperdrive.sampling import RandomParameterSampling
from azureml.train.hyperdrive.runconfig import HyperDriveConfig
from azureml.train.hyperdrive.parameter_expressions import uniform
import os

# Specify parameter sampler
# randomized parameter sampling used for resource conservative assessment of parameter space
ps = RandomParameterSampling(
    {
        "--C": uniform(0.1, 10),
        "--max_iter": uniform(100, 5000)
    }
)

# Specify a Policy
# apply bandit policy to terminate if any subsequent run after the 6th run (tested every other run) 
# is less than 1/1.2 (83%) of the currently best performing run
policy = BanditPolicy(evaluation_interval=2, slack_factor=0.2, delay_evaluation=6)



# Create a SKLearn estimator for use with train.py
est = SKLearn(
    source_directory="./training",
    compute_target = cpu_cluster,
    pip_packages=['azureml-sdk','argparse', ''],
    conda_packages=['pandas','ipykernel', 'numpy'],
    entry_script='train.py'
)

# Create a HyperDriveConfig using the estimator, hyperparameter sampler, and policy.
hyperdrive_config = HyperDriveConfig(
    estimator=est,
    hyperparameter_sampling=ps,
    policy=policy,
    primary_metric_name='Accuracy',
    primary_meteic_goal=PrimaryMetricGoal.MAXIMIZE,
    max_total_runs=12,
    max_concurrent_runs=4

)

In [ ]:
# Submit your hyperdrive run to the experiment and show run details with the widget.
hdr = exp.submit(config=hyperdrive_config)
RunDetails(hdr).show()

In [ ]:
import joblib
# Get your best run and save the model from that run.
best_run_hdr = hdr.get_best_run_by_primary_metric()
best_run_metrics = best_run_hdr.get_metrics()
parameter_values = best_run_hdr.get_details()['runDefinition']['arguments']

print('run_id: ', best_run_hdr.id)
print(' Accuracy:', best_run_metrics['Accuracy'])
print(' Parameter_Values:',parameter_values)
os.makedirs('models', exist_ok=True)
joblib.dump(value=best_run_hdr, filename='./models/hdr_model.pkl')

In [ ]:
from azureml.data.dataset_factory import TabularDatasetFactory

# Create TabularDataset using TabularDatasetFactory
# Data is available at: 
# "https://automlsamplenotebookdata.blob.core.windows.net/automl-sample-notebook-data/bankmarketing_train.csv"

datastore_path = "https://automlsamplenotebookdata.blob.core.windows.net/automl-sample-notebook-data/bankmarketing_train.csv"
ds = TabularDatasetFactory.from_delimited_files(path=datastore_path, support_multi_line=True)

In [ ]:
from train import clean_data

# Use the clean_data function to clean your data.
x, y = clean_data(ds)
x['y'] = y

In [ ]:
from azureml.train.automl import AutoMLConfig

# Set parameters for AutoMLConfig
# NOTE: DO NOT CHANGE THE experiment_timeout_minutes PARAMETER OR YOUR INSTANCE WILL TIME OUT.
# If you wish to run the experiment longer, you will need to run this notebook in your own
# Azure tenant, which will incur personal costs.
automl_config = AutoMLConfig(
    experiment_timeout_minutes=30,
    task='classification',
    primary_metric='Accuracy',
    training_data=x,
    label_column_name='y',
    n_cross_validations=5)

In [2]:
# Submit your automl run
exp = Experiment(ws, 'udacity-project-automl')
automl_run = exp.submit(automl_config, show_output=True)
RunDetails(automl_run).show()

In [ ]:
# Retrieve and save your best automl model.
best_run_automl, fitted_model = automl_run.get_output()
best_run_metrics = best_run_automl.get_metrics()
for metric_name in best_run_metrics:
    metric = best_run_metrics[metric_name]
    print(metric_name, metric)
# save model in pickle file
joblib.dump(value=fitted_model, filename='./models/automl_model.pkl')
best_run_automl.register_model(model_path='./models/automl_model.pkl', model_name='automl_model')